In [79]:
import pandas as pd
import numpy as np

In [80]:
path = 'D:/мисис/питон-1/MISIS_2022/season_2/data/DataSet_w_NA.xlsx'
df_init = pd.read_excel(path, sheet_name="Испорченные факты")
df = df_init
df

,Номер заказа,Возврат,Дата заказа,Дата отправки,Режим отправки,Факты.Контрагент ID,Факты.Товар ID,"Продажи, руб","Продажи, шт",Скидка,Маржинальная прибыль,Повторение заказа,Повторение контрагента,Повторение товара
0,2014-100279,Нет,2015-09-10 00:00:00,2015-09-14 00:00:00,Стандарт,CLNT0000422,PROD0000870,11190.0,2.0,0.0,5371.20,1,7,3
1,2014-100293,Нет,2015-09-14 00:00:00,2015-09-18 00:00:00,Стандарт,CLNT0000784,PROD0000646,45528.0,6.0,0.2,15934.80,1,4,8
2,2014-100328,Нет,2015-07-28 00:00:00,2015-08-03 00:00:00,Стандарт,CLNT0000342,PROD0000187,1964.0,1.0,0.2,662.85,1,4,12
3,2014-100363,Нет,2015-10-08 00:00:00,2015-10-15 00:00:00,Стандарт,CLNT0000146,PROD0001095,1184.0,2.0,0.2,414.40,2,12,7
4,2014-100363,Нет,2015-10-08 00:00:00,2015-10-15 00:00:00,Стандарт,CLNT0000146,PROD0001616,9504.0,3.0,0.2,3445.20,2,12,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10288,2018-169929,Нет,2019-03-25 00:00:00,2019-03-28 00:00:00,Первый класс,CLNT0000182,PROD0000869,19950.0,5.0,0.0,5187.00,1,14,6
10289,2018-169978,Нет,2019-06-24 00:00:00,2019-06-28 00:00:00,Стандарт,CLNT0000575,PROD0000293,135882.0,2.0,0.1,24156.80,2,11,11
10290,2018-169978,Нет,2019-06-24 00:00:00,2019-06-28 00:00:00,Стандарт,CLNT0000575,PROD0001131,7188.0,3.0,0.2,2425.95,2,11,4
10291,2018-169999,Нет,2018-12-19 00:00:00,2018-12-26 00:00:00,Стандарт,CLNT0000013,PROD0000549,5784.0,3.0,0.2,1879.80,2,11,13


In [139]:
df_init['Режим отправки'].unique()

array(['Стандарт', 'Второй класс', 'Первый класс', 'В тот же день'],
      dtype=object)

Используя RandomForestClassifier и еще 2 метода МО предсказать будущий ABC класс на основе любых фичей

Найти причину/фактор низкой маржинальности

In [81]:
df = df.dropna()
df =pd.pivot_table(df, values=['Продажи, руб', 'Продажи, шт','Повторение заказа','Маржинальная прибыль','Повторение товара'], index=["Факты.Товар ID"],
                     aggfunc={'Продажи, шт': [np.median, np.sum],
                              'Продажи, руб': np.sum,
                              'Повторение заказа': np.sum,
                              'Маржинальная прибыль': np.sum
                               })
newname=df.columns.map('_'.join)
df.columns=newname
df=df.reset_index()
df['Продажи, шт_sum']

0       12.0
1       51.0
2       23.0
3       29.0
4       24.0
        ... 
1857     3.0
1858     3.0
1859     4.0
1860     5.0
1861     3.0
Name: Продажи, шт_sum, Length: 1862, dtype: float64

In [83]:
total_sale=df['Продажи, руб_sum'].sum()
df['Доля']=df['Продажи, руб_sum']/total_sale * 100
df['Доля'].sum()
df = df.sort_values(by=('Продажи, руб_sum'), ascending=False)
df=df.assign(sum_d=df['Доля'].cumsum())
df.loc[(df['sum_d'] <= 80), 'ABC'] = 'A'
df.loc[(df['sum_d'] > 80) & (df['sum_d'] <= 95), 'ABC'] = 'B'
df.loc[(df['sum_d'] > 95), 'ABC'] = 'C'

In [84]:
df

,Факты.Товар ID,Маржинальная прибыль_sum,Повторение заказа_sum,"Продажи, руб_sum","Продажи, шт_median","Продажи, шт_sum",Доля,sum_d,ABC
1388,PROD0001406,1.259996e+07,17,30799912.0,4.0,20.0,2.681845,2.681845,A
450,PROD0000454,3.876520e+06,23,13726692.0,3.0,31.0,1.195226,3.877071,A
1411,PROD0001430,-9.055392e+05,7,11319240.0,6.0,6.0,0.985602,4.862673,A
483,PROD0000488,1.746230e-10,15,10935288.0,5.0,39.0,0.952170,5.814843,A
1176,PROD0001187,1.116753e+06,33,9911739.5,3.0,37.0,0.863046,6.677889,A
...,...,...,...,...,...,...,...,...,...
1838,PROD0001871,1.578500e+03,1,3850.0,2.0,2.0,0.000335,99.999089,C
1854,PROD0001887,1.193400e+03,1,3536.0,2.0,2.0,0.000308,99.999396,C
1747,PROD0001779,1.555200e+03,3,3240.0,1.0,1.0,0.000282,99.999679,C
1675,PROD0001703,1.411200e+03,1,2880.0,2.0,2.0,0.000251,99.999929,C


In [85]:
df_abc = df[['ABC', 'Факты.Товар ID']]

In [86]:
df_merge = pd.merge(df_init,df_abc,on='Факты.Товар ID')

In [141]:
df_merge

,Номер заказа,Возврат,Дата заказа,Дата отправки,Режим отправки,Факты.Контрагент ID,Факты.Товар ID,"Продажи, руб","Продажи, шт",Скидка,Маржинальная прибыль,Повторение заказа,Повторение контрагента,Повторение товара,ABC
0,2014-100279,Нет,2015-09-10 00:00:00,2015-09-14 00:00:00,Стандарт,CLNT0000422,PROD0000870,11190.0,2.0,0.0,5371.2,1,7,3,C
1,2015-146283,Нет,2016-03-08 00:00:00,2016-03-15 00:00:00,Стандарт,CLNT0000472,PROD0000870,8952.0,2.0,0.2,3133.2,3,14,3,C
2,2015-164210,Нет,2016-05-18 00:00:00,2016-05-20 00:00:00,Второй класс,CLNT0000562,PROD0000870,17904.0,4.0,0.2,6266.4,2,12,3,C
3,2014-100293,Нет,2015-09-14 00:00:00,2015-09-18 00:00:00,Стандарт,CLNT0000784,PROD0000646,45528.0,6.0,0.2,15934.8,1,4,8,B
4,2016-124653,Нет,2017-06-24 00:00:00,2017-06-26 00:00:00,Первый класс,CLNT0000032,PROD0000646,66395.0,7.0,0.0,31869.6,3,14,8,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9988,2018-161970,Нет,2019-03-22 00:00:00,2019-03-27 00:00:00,Стандарт,CLNT0000394,PROD0001844,8970.0,3.0,0.0,3229.2,2,5,1,C
9989,2018-165323,Нет,2018-12-17 00:00:00,2018-12-21 00:00:00,Стандарт,CLNT0000683,PROD0001766,50670.0,3.0,0.0,4053.6,2,12,2,C
9990,2018-166856,Нет,2019-03-18 00:00:00,2019-03-22 00:00:00,Стандарт,CLNT0000603,PROD0001766,50670.0,3.0,0.0,4053.6,2,14,2,C
9991,2018-165323,Нет,2018-12-17 00:00:00,2018-12-21 00:00:00,Стандарт,CLNT0000683,PROD0001862,1702250.0,5.0,0.0,834102.5,2,12,1,A


In [150]:
allinput = df_merge[['Возврат', 'Повторение заказа', 'Повторение контрагента', 'Повторение товара', 'Режим отправки', 'Продажи, руб']]
alloutput = df_merge[["ABC"]]

In [151]:
allinput['Возврат']  = allinput['Возврат'].map({'Да': 1, 'Нет': 0})
allinput['Режим отправки'] = allinput['Режим отправки'].map({'Стандарт': 0, 'Второй класс': 1, 'Первый класс': 2, 'В тот же день': 3})
#alloutput['ABC'] = alloutput['ABC'].map({'A': 2, 'B': 1, 'C': 0})

C:\Users\ikoly\AppData\Local\Temp/ipykernel_11688/646126928.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  allinput['Возврат']  = allinput['Возврат'].map({'Да': 1, 'Нет': 0})
C:\Users\ikoly\AppData\Local\Temp/ipykernel_11688/646126928.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  allinput['Режим отправки'] = allinput['Режим отправки'].map({'Стандарт': 0, 'Второй класс': 1, 'Первый класс': 2, 'В тот же день': 3})


In [152]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(allinput, alloutput, test_size=0.2)

In [153]:
X_train

,Возврат,Повторение заказа,Повторение контрагента,Повторение товара,Режим отправки,"Продажи, руб"
5153,0,2,18,6,0,20975.0
2726,0,3,10,6,2,9760.0
6234,0,3,20,7,0,171960.0
5528,0,5,10,6,0,43152.0
6939,0,3,23,8,1,12000.0
...,...,...,...,...,...,...
4918,0,2,5,5,0,39720.0
9553,0,2,25,6,0,92940.0
2127,0,1,17,8,2,3270.0
7123,0,5,14,7,3,27112.0


In [154]:
Y_train

,ABC
5153,C
2726,C
6234,A
5528,B
6939,B
...,...
4918,B
9553,B
2127,C
7123,B


In [155]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
rfc = RandomForestClassifier()
rfc.fit(X_train, Y_train.values.ravel())
display(rfc.score(X_train, Y_train))

0.9998749061796347

In [156]:
rfc.predict(X_test)

array(['A', 'B', 'A', ..., 'B', 'C', 'C'], dtype=object)

In [157]:
from sklearn.metrics import classification_report

print(classification_report(Y_test, rfc.predict(X_test)))

              precision    recall  f1-score   support

           A       0.84      0.84      0.84       505
           B       0.72      0.69      0.70       590
           C       0.88      0.90      0.89       904

    accuracy                           0.82      1999
   macro avg       0.81      0.81      0.81      1999
weighted avg       0.82      0.82      0.82      1999



In [158]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import recall_score
gbc = GradientBoostingClassifier()
gbc.fit(X_train, Y_train.values.ravel())
display(gbc.score(X_train, Y_train))

0.8432574430823118

In [159]:
print(classification_report(Y_test, gbc.predict(X_test)))

              precision    recall  f1-score   support

           A       0.83      0.87      0.85       505
           B       0.74      0.68      0.71       590
           C       0.89      0.91      0.90       904

    accuracy                           0.83      1999
   macro avg       0.82      0.82      0.82      1999
weighted avg       0.83      0.83      0.83      1999



In [160]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import recall_score
abc = AdaBoostClassifier()
abc.fit(X_train, Y_train.values.ravel())
display(abc.score(X_train, Y_train))

0.8149862396797598

In [161]:
print(classification_report(Y_test, abc.predict(X_test)))

              precision    recall  f1-score   support

           A       0.78      0.88      0.83       505
           B       0.72      0.60      0.65       590
           C       0.88      0.91      0.89       904

    accuracy                           0.81      1999
   macro avg       0.79      0.80      0.79      1999
weighted avg       0.81      0.81      0.81      1999

